# Recommender Systems

## Import Libraries

In [40]:
import pandas as pd
import numpy as np

## Import Dataset

In [41]:
# users_file = codecs.open('./CSVs/BX-Users.csv', mode='r', encoding='utf8')
# books_file = codecs.open('./CSVs/BX-Books.csv', mode='r', encoding='utf8')
# ratings_file = codecs.open('./CSVs/BX-Book-Ratings.csv', mode='r', encoding='utf8')

users = pd.read_csv('./CSVs/BX-Users.csv', error_bad_lines=False, sep=';', encoding="latin1")
books = pd.read_csv('./CSVs/BX-Books.csv', error_bad_lines=False, sep=';', encoding="latin1")
ratings = pd.read_csv('./CSVs/BX-Book-Ratings.csv', error_bad_lines=False, sep=';', encoding="latin1")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
c:\users\zomorrod\appdata\local\programs\python\python38-32\lib\site-packa

In [42]:
print('The shape of Ratings is: ' + str(ratings.shape))
print('The shape of Users is: ' + str(users.shape))
print('The shape of Books is: ' + str(books.shape))

The shape of Ratings is: (1149780, 3)
The shape of Users is: (278858, 3)
The shape of Books is: (271360, 8)


## Data Preprocessing

In [43]:
users.columns = ['userId', 'location', 'age']
users.head()

,userId,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [44]:
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageURLS', 'imageURLM', 'imageURLL']
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageURLS,imageURLM,imageURLL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [45]:
ratings.columns = ['userId', 'ISBN', 'bookRating']
ratings.head()

,userId,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [46]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

features = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageURLS', 'imageURLM', 'imageURLL']
for feature in features:
    books[feature]=books[feature].fillna('')

def combined_features(row):
    return row['bookTitle']+" "+row['bookAuthor']+" "+str(row['yearOfPublication'])+" "+row['publisher']+" "+row['imageURLS']+" "+row['imageURLM']+" "+" "+row['imageURLS']

books["combined_features"] = books.apply(combined_features, axis =1)
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageURLS,imageURLM,imageURLL,combined_features
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Classical Mythology Mark P. O. Morford 2002 Ox...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,Clara Callan Richard Bruce Wright 2001 HarperF...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,Decision in Normandy Carlo D'Este 1991 HarperP...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,The Mummies of Urumchi E. J. W. Barber 1999 W....


In [52]:
# cv = CountVectorizer()
# count_matrix = cv.fit_transform(books["combined_features"])
# print(count_matrix)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
# tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tf = TfidfVectorizer()
x = books["combined_features"]
tfidf_matrix = tf.fit_transform(x)

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 
results = {}
for idx, row in ds.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
   similar_items = [(cosine_similarities[idx][i], ds['id'][i]) for i in similar_indices] 
   results[row['id']] = similar_items[1:]

TypeError: __init__() missing 2 required positional arguments: 'shape' and 'dtype'

## 1. Collaborative Filtering

In [48]:
# from sklearn.metrics.pairwise import cosine_similarity

# # Change chunk_size to control resource consumption and speed
# # Higher chunk_size means more memory/RAM needed but also faster 
# chunk_size = 100 
# matrix_len = count_matrix.shape[0] # Not sparse numpy.ndarray

# def similarity_cosine_by_chunk(start, end):
#     if end > matrix_len:
#         end = matrix_len
#     return cosine_similarity(X=count_matrix[start:end], Y=count_matrix) # scikit-learn function

# for chunk_start in range(0, matrix_len, chunk_size):
#     cosine_similarity_chunk = similarity_cosine_by_chunk(chunk_start, chunk_start+chunk_size)
#     print(cosine_similarity_chunk)
#     # Handle cosine_similarity_chunk  ( Write it to file_timestamp and close the file )
#     # Do not open the same file again or you may end up with out of memory after few chunks 
    
    
# # count_matrix = count_matrix.astype(np.float32)
# # cosine_sim = cosine_similarity(count_matrix)
# # print(cosine_sim)

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

A_sparse = sparse.csr_matrix(count_matrix)

similarities = cosine_similarity(A_sparse)
print('pairwise dense output:\n {}\n'.format(similarities))

#also can output sparse matrices
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)
print('pairwise sparse output:\n {}\n'.format(similarities_sparse))

MemoryError: Unable to allocate 40.8 MiB for an array with shape (5353546,) and data type float64

## 2. Content-based Filtering

## Resources

1. https://stackoverflow.com/questions/18039057/python-pandas-error-tokenizing-data
2. https://blog.guillaumeagis.eu/recommendation-algorithms-with-apache-mahout/
3. https://towardsdatascience.com/using-cosine-similarity-to-build-a-movie-recommendation-system-ae7f20842599